# Cas pratique en réseau



**Table of contents**<a id='toc0_'></a>    
- [Introduction](#toc1_)    
- [Cas 1 : newtorkx](#toc2_)    
  - [Description](#toc2_1_)    
  - [Algorithmes utilisés](#toc2_2_)    
    - [Dijkstra](#toc2_2_1_)    
    - [A*](#toc2_2_2_)    
    - [Floyd-Warshall](#toc2_2_3_)    
    - [Bellman-Ford](#toc2_2_4_)    
- [Cas 2 : OSPF](#toc3_)    
  - [Description](#toc3_1_)    
  - [Source :](#toc3_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=4
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_'></a>[Introduction](#toc0_)

Description de ce qu'on va faire

## <a id='toc2_'></a>[Cas 1 : newtorkx](#toc0_)

### <a id='toc2_1_'></a>[Description](#toc0_)

NetworkX est une bibliothèque Python utilisée pour la création, la manipulation et l'étude de structures de graphes et réseaux complexes. Elle offre des outils pour représenter des graphes sous forme de structures de données (orientés, non orientés, pondérés, etc.) et propose des algorithmes avancés pour analyser leurs propriétés, comme les plus courts chemins, la centralité, la connectivité, et bien plus.

Github networkx : https://github.com/networkx/networkx

### <a id='toc2_2_'></a>[Algorithmes utilisés](#toc0_)

Networkx propose plusieurs algorithmes pour travailler sur les graphes : 



In [7]:
import networkx as nx

# Création du graphe pondéré
G = nx.DiGraph()
G.add_weighted_edges_from([("A", "B", 1), ("B", "C", 2), ("A", "C", 4), ("C", "D", 1)])


#### <a id='toc2_2_1_'></a>[Dijkstra](#toc0_)

L'algorithme de Dijkstra est utilisé pour trouver le plus court chemin dans un graphe pondéré avec des poids positifs.
Il fonctionne en explorant systématiquement les chemins les moins coûteux depuis le sommet de départ, garantissant la
solution optimale.


In [8]:
# Plus court chemin de A à D
shortest_path = nx.dijkstra_path(G, source="A", target="D", weight="weight")
print("Plus court chemin (Dijkstra) :", shortest_path)

Plus court chemin (Dijkstra) : ['A', 'B', 'C', 'D']



#### <a id='toc2_2_2_'></a>[A*](#toc0_)

L'algorithme A* améliore Dijkstra en utilisant une fonction heuristique qui estime la distance restante jusqu'à la cible.
Cela permet de réduire le nombre de nœuds explorés et accélère la recherche du chemin optimal.


In [9]:
# Heuristique basée sur une distance fictive
def heuristic(u, v):
    return abs(ord(u) - ord(v))  # Différence simple des codes ASCII

# Plus court chemin utilisant A*
a_star_path = nx.astar_path(G, source="A", target="D", heuristic=heuristic, weight="weight")
print("Plus court chemin (A*) :", a_star_path)


Plus court chemin (A*) : ['A', 'B', 'C', 'D']



#### <a id='toc2_2_3_'></a>[Floyd-Warshall](#toc0_)

L'algorithme de Floyd-Warshall trouve les plus courts chemins entre tous les couples de nœuds dans un graphe pondéré.
Il utilise une approche dynamique, mise à jour progressive des distances via des sommets intermédiaires.

**Utilisation réelle dans les réseaux** :
- Calcul des chemins optimaux entre tous les pairs de routeurs dans un réseau.
- Analyse de la connectivité dans les réseaux sociaux.


In [10]:
# Matrice des plus courts chemins pour tous les nœuds
shortest_paths_matrix = dict(nx.floyd_warshall(G, weight="weight"))
print("Plus courts chemins (Floyd-Warshall) :", shortest_paths_matrix)

Plus courts chemins (Floyd-Warshall) : {'A': defaultdict(<function floyd_warshall_predecessor_and_distance.<locals>.<lambda>.<locals>.<lambda> at 0x0000019035E13250>, {'A': 0, 'B': 1, 'C': 3, 'D': 4}), 'B': defaultdict(<function floyd_warshall_predecessor_and_distance.<locals>.<lambda>.<locals>.<lambda> at 0x0000019035E135B0>, {'B': 0, 'C': 2, 'A': inf, 'D': 3}), 'C': defaultdict(<function floyd_warshall_predecessor_and_distance.<locals>.<lambda>.<locals>.<lambda> at 0x0000019035E136D0>, {'C': 0, 'D': 1, 'A': inf, 'B': inf}), 'D': defaultdict(<function floyd_warshall_predecessor_and_distance.<locals>.<lambda>.<locals>.<lambda> at 0x0000019035E137F0>, {'D': 0, 'A': inf, 'B': inf, 'C': inf})}



#### <a id='toc2_2_4_'></a>[Bellman-Ford](#toc0_)

L'algorithme de Bellman-Ford calcule le plus court chemin dans les graphes où les poids peuvent être négatifs.
Il explore tous les chemins possibles et mise à jour les coûts minimaux.


In [11]:
# Plus court chemin avec Bellman-Ford
shortest_path_bf = nx.bellman_ford_path(G, source="A", target="D", weight="weight")
print("Plus court chemin (Bellman-Ford) :", shortest_path_bf)

Plus court chemin (Bellman-Ford) : ['A', 'B', 'C', 'D']


## <a id='toc3_'></a>[Cas 2 : OSPF](#toc0_)

### <a id='toc3_1_'></a>[Description](#toc0_)

OSPF (Open Shortest Path First) est un protocole de routage d’état de liaison. Il est conçu pour être utilisé au sein d’un seul et même système autonome. Chaque routeur OSPF maintient une base de données identique décrivant la topologie du système dont il fait partie. À partir de cette base de données, une table de routage est calculée en construisant un arbre des chemins les plus courts.





### Exemple d'utilisation de Dijkstra dans OSPF

In [ ]:
# Code d'exemple a faire

### <a id='toc3_2_'></a>[Source :](#toc0_)

La version actuelle la plus utilisé est OSPFv2, définie dans la RFC 2328.

La RFC de OSPF : https://www.ietf.org/rfc/rfc2328.txt

RFC (Request For Comments) : Les RFC sont des documents qui décrivent les protocoles, les procédures, les programmes et les concepts qui constituent Internet. Ils sont écrits et publiés par l'Internet Engineering Task Force (IETF).
